## DB에서 CSV로 추출
- rental_daily_station 테이블에서 rental_station_id 별로 rental_count 합산

---

In [3]:
import mysql.connector
import pandas as pd

# MySQL RDS 연결
conn = mysql.connector.connect(
    host='jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com',
    port=3306,
    user='root',
    password='jinhyuk2dacibul',
    database='project1'
)

# 쿼리 실행: rental_station_id 별로 rental_count 합산
query = """
SELECT rental_station_id, SUM(rental_count) AS total_rental_count
FROM rental_daily_station
GROUP BY rental_station_id;
"""

# Pandas DataFrame에 쿼리 결과 로드
df = pd.read_sql(query, conn)

# DataFrame을 CSV 파일로 저장
df.to_csv('../data/rental_station_summary.csv', index=False)

# 연결 종료
conn.close()

print("CSV 파일로 저장이 완료되었습니다.")


/tmp/ipykernel_34349/698246760.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CSV 파일로 저장이 완료되었습니다.


---
## CSV가공 후 추출 - 시간대별 버스 승객 수
- **bus_hourly_summary_cleaned** CSV 파일 에서 **date_period**와 **hour_of_day** 기준으로 **bus_count**를 계산하여 **CSV** 파일로 저장합니다.
- 이 과정에서는 **hour_of_day**별로 승객 수를 **합산**하여 시간대별 **버스 승객 수**를 구하고, 그 결과를 새로운 **CSV** 파일로 저장합니다.
---

In [12]:
import pandas as pd

# 데이터 불러오기
df_bus = pd.read_csv("../data/bus_hourly_summary_cleaned.csv")

# 시간대별로 합산할 컬럼들
boarding_columns = [f"boarding_{i:02d}" for i in range(24)]
alighting_columns = [f"alighting_{i:02d}" for i in range(24)]

# 시간대별 총 승객 수 계산
df_bus['bus_count'] = df_bus[boarding_columns].sum(axis=1) + df_bus[alighting_columns].sum(axis=1)

# 'hour_of_day'와 'bus_count' 추출하여 새로운 형식으로 변환
df_bus_long = pd.melt(df_bus, 
                      id_vars=['date_period'],  # 'dong'과 'gu' 제거, 'date_period' 유지
                      value_vars=boarding_columns + alighting_columns, 
                      var_name='hour_of_day', 
                      value_name='count')

# 'hour_of_day'에서 시간 정보만 추출 (예: boarding_00 -> 0)
df_bus_long['hour_of_day'] = df_bus_long['hour_of_day'].apply(lambda x: int(x.split('_')[1]))

# 'count' 컬럼을 버스 승객수로 지정 (boarding과 alighting 구분 없이 합산된 총승객수)
df_bus_long['bus_count'] = df_bus_long['count']

# date_period와 hour_of_day별 총합 계산
df_bus_summary = df_bus_long.groupby(['date_period', 'hour_of_day'])['bus_count'].sum().reset_index()

# CSV로 저장
df_bus_summary.to_csv("../data/bus_hourly_summary_total_with_date_period.csv", index=False)

print("파일 저장 완료")


파일 저장 완료


---
## CSV가공 후 추출 - 시간대별 지하철 승객 수
- **subway_hourly_summary_cleaned** CSV 파일에서 **date_period**와 **hour_of_day** 기준으로 **subway_count**를 계산하여 **CSV** 파일로 저장합니다.
- 이 과정에서는 **hour_of_day**별로 승객 수를 **합산**하여 시간대별 **지하철 승객 수**를 구하고, 그 결과를 새로운 **CSV** 파일로 저장합니다.
---


In [13]:
import pandas as pd

# 데이터 불러오기
df_subway = pd.read_csv("../data/subway_hourly_summary_cleaned.csv")

# 시간대별로 합산할 컬럼들
boarding_columns = [f"boarding_{i:02d}" for i in range(24)]
alighting_columns = [f"alighting_{i:02d}" for i in range(24)]

# 시간대별 총 승객 수 계산
df_subway['subway_count'] = df_subway[boarding_columns].sum(axis=1) + df_subway[alighting_columns].sum(axis=1)

# 'hour_of_day'와 'subway_count' 추출하여 새로운 형식으로 변환
df_subway_long = pd.melt(df_subway, 
                         id_vars=['date_period'],  # 'dong'과 'gu' 제거, 'date_period' 유지
                         value_vars=boarding_columns + alighting_columns, 
                         var_name='hour_of_day', 
                         value_name='count')

# 'hour_of_day'에서 시간 정보만 추출 (예: boarding_00 -> 0)
df_subway_long['hour_of_day'] = df_subway_long['hour_of_day'].apply(lambda x: int(x.split('_')[1]))

# 'count' 컬럼을 지하철 승객수로 지정 (boarding과 alighting 구분 없이 합산된 총승객수)
df_subway_long['subway_count'] = df_subway_long['count']

# date_period와 hour_of_day별 총합 계산
df_subway_summary = df_subway_long.groupby(['date_period', 'hour_of_day'])['subway_count'].sum().reset_index()

# CSV로 저장
df_subway_summary.to_csv("../data/subway_hourly_summary_total_with_date_period.csv", index=False)

print("파일 저장 완료")


파일 저장 완료


---
## CSV 가공 후 추출 - 행정동 별 대중교통 이용수 합
- **subway_hourly_summary_cleaned, bus_hourly_summary_cleaned** CSV 파일에서 **date_period**와 **hour_of_day** 기준으로 **subway_count, bus_count**를 계산하여 **CSV** 파일로 저장합니다.
  
- 이 과정에서는 **hour_of_day**별로 승객 수를 **합산**하여 시간대별 **승객 수**를 구하고, 그 결과를 새로운 **CSV** 파일로 저장합니다.

---

In [1]:
import pandas as pd

# 데이터 불러오기
df_bus = pd.read_csv("../data/bus_hourly_summary_cleaned.csv")
df_subway = pd.read_csv("../data/subway_hourly_summary_cleaned.csv")

# 버스 시간대별로 합산할 컬럼들
bus_boarding_columns = [f"boarding_{i:02d}" for i in range(24)]
bus_alighting_columns = [f"alighting_{i:02d}" for i in range(24)]

# 지하철 시간대별로 합산할 컬럼들
subway_boarding_columns = [f"boarding_{i:02d}" for i in range(24)]
subway_alighting_columns = [f"alighting_{i:02d}" for i in range(24)]

# 버스와 지하철의 총 이용객 수 계산
df_bus['bus_count'] = df_bus[bus_boarding_columns].sum(axis=1) + df_bus[bus_alighting_columns].sum(axis=1)
df_subway['subway_count'] = df_subway[subway_boarding_columns].sum(axis=1) + df_subway[subway_alighting_columns].sum(axis=1)

# 행정동별 버스와 지하철 이용객 수 합산
bus_agg_df = df_bus.groupby(['dong', 'gu'])['bus_count'].sum().reset_index()
subway_agg_df = df_subway.groupby(['dong', 'gu'])['subway_count'].sum().reset_index()

# 결과 확인
print("버스 이용객 수 합산 결과:")
print(bus_agg_df.head())

print("지하철 이용객 수 합산 결과:")
print(subway_agg_df.head())

# CSV로 저장 (../data 경로)
bus_agg_df.to_csv("../data/bus_usage_summary.csv", index=False)
subway_agg_df.to_csv("../data/subway_usage_summary.csv", index=False)

print("파일 저장 완료")


버스 이용객 수 합산 결과:
   dong   gu   bus_count
0   가락동  송파구  16311776.0
1  가리봉동  구로구   5349066.0
2   가산동  금천구  21710342.0
3   가양동  강서구   5654045.0
4   가회동  종로구    509478.0
지하철 이용객 수 합산 결과:
   dong   gu  subway_count
0  가락2동  송파구    28637156.0
1   가산동  금천구    27801649.0
2  가흥1동  송파구    14709154.0
3   갈현동  은평구    23717223.0
4  개포3동  강남구    10532482.0
파일 저장 완료


## DB에서 CSV로 추출
- **bus_station_hourly_timeflow** 테이블에서 각 버스정류소의 연간 총 이용객 수를 합산하고, 
- **bus_station_location** 테이블로부터 해당 버스정류소의 **dong**과 **gu** 정보를 추출하여 **CSV** 파일로 저장합니다.

---

In [2]:
import mysql.connector
import pandas as pd

# MySQL 연결 설정
db_connection = mysql.connector.connect(
    host='jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com',
    port=3306,
    user='root',
    password='jinhyuk2dacibul',
    database='project1'
)

# 커서 생성
cursor = db_connection.cursor()

# SQL 쿼리 작성: 각 버스정류소의 연간 총 이용객 수 합산 및 동, 구 추출
query = """
    SELECT b.dong, b.gu, SUM(bt.boarding_00 + bt.alighting_00 + bt.boarding_01 + bt.alighting_01 + 
                             bt.boarding_02 + bt.alighting_02 + bt.boarding_03 + bt.alighting_03 + 
                             bt.boarding_04 + bt.alighting_04 + bt.boarding_05 + bt.alighting_05 + 
                             bt.boarding_06 + bt.alighting_06 + bt.boarding_07 + bt.alighting_07 + 
                             bt.boarding_08 + bt.alighting_08 + bt.boarding_09 + bt.alighting_09 + 
                             bt.boarding_10 + bt.alighting_10 + bt.boarding_11 + bt.alighting_11 + 
                             bt.boarding_12 + bt.alighting_12 + bt.boarding_13 + bt.alighting_13 + 
                             bt.boarding_14 + bt.alighting_14 + bt.boarding_15 + bt.alighting_15 + 
                             bt.boarding_16 + bt.alighting_16 + bt.boarding_17 + bt.alighting_17 + 
                             bt.boarding_18 + bt.alighting_18 + bt.boarding_19 + bt.alighting_19 + 
                             bt.boarding_20 + bt.alighting_20 + bt.boarding_21 + bt.alighting_21 + 
                             bt.boarding_22 + bt.alighting_22 + bt.boarding_23 + bt.alighting_23) AS total_usage
    FROM bus_station_hourly_timeflow bt
    JOIN bus_station_location b ON bt.bus_station_id = b.bus_station_id
    GROUP BY b.dong, b.gu
"""

# 쿼리 실행
cursor.execute(query)

# 결과를 pandas DataFrame으로 저장
data = cursor.fetchall()
columns = ['dong', 'gu', 'total_usage']
df = pd.DataFrame(data, columns=columns)

# 결과를 CSV 파일로 저장
df.to_csv('../data/bus_station_usage_summary.csv', index=False)

# 데이터베이스 연결 종료
cursor.close()
db_connection.close()

print("CSV 파일로 저장 완료")


CSV 파일로 저장 완료


---
## CSV 가공 후 추출 - 법정동(dong)을 행정동으로 매핑하여 '행정동' 컬럼으로 추가
- bus_hourly_summary_cleaned.csv 의 dong = 법정동
- 행정구역(동별)_20250325154718.csv 에서 법정동 ↔ 행정동 관계 추출
- 법정동 기준으로 매핑하여, '행정동' 컬럼을 추가
---

In [3]:
import pandas as pd

# 1. 파일 로드
bus_df = pd.read_csv("../data/bus_hourly_summary_cleaned.csv", encoding="utf-8-sig")
admin_df = pd.read_csv("../data/행정구역(동별)_20250325154718.csv", encoding="utf-8-sig", skiprows=2)

# 2. 법정동-행정동 매핑 테이블 생성
# 동별(2) = 행정동명, 동별(3) = 법정동명
mapping_df = admin_df[["동별(2)", "동별(3)"]].copy()
mapping_df.columns = ["행정동", "법정동"]

# 중복 제거 (법정동 이름이 중복될 수 있으니 하나만 선택)
mapping_df = mapping_df.drop_duplicates(subset="법정동", keep="first")

# 3. 매핑: bus_df의 dong(=법정동) 기준으로 행정동 매핑
bus_df = pd.merge(bus_df, mapping_df, how="left", left_on="dong", right_on="법정동")

# 4. 매핑에 사용한 '법정동' 컬럼 제거 (dong은 그대로 유지)
bus_df = bus_df.drop(columns=["법정동"])

# 5. 결과 저장
bus_df.to_csv("../data/bus_hourly_summary_with_admin_column.csv", index=False, encoding="utf-8-sig")


---
## CSV 추출 - MySQL에서 bus_station_hourly_timeflow 테이블 전체 조회 후 저장
- MySQL의 project1 DB에서 bus_station_hourly_timeflow 테이블 전체 조회
- pandas를 이용해 DataFrame으로 불러온 후 CSV로 저장
- 저장 경로: ../data/bus_station_hourly_timeflow.csv
---


In [4]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결 정보
conn = mysql.connector.connect(
    host='jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com',
    port=3306,
    user='root',
    password='jinhyuk2dacibul',
    database='project1'
)

# 2. 데이터 불러오기
query = "SELECT * FROM bus_station_hourly_timeflow"
df = pd.read_sql(query, conn)

# 3. 저장 경로 및 파일명
save_path = '../data/bus_station_hourly_timeflow.csv'

# 4. CSV 저장
df.to_csv(save_path, index=False, encoding='utf-8-sig')

# 5. 연결 종료
conn.close()

print("CSV 저장 완료:", save_path)


/tmp/ipykernel_26024/1530841957.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CSV 저장 완료: ../data/bus_station_hourly_timeflow.csv


---
## CSV 추출 - MySQL에서 bus_station_location 테이블 전체 조회 후 저장
- MySQL의 project1 DB에서 bus_station_location 테이블 전체 조회
- pandas를 이용해 DataFrame으로 불러온 후 CSV로 저장
- 저장 경로: ../data/bus_station_location.csv
---


In [5]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결
conn = mysql.connector.connect(
    host='jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com',
    port=3306,
    user='root',
    password='jinhyuk2dacibul',
    database='project1'
)

# 2. 테이블 조회
query = "SELECT * FROM bus_station_location"
df = pd.read_sql(query, conn)

# 3. CSV 저장
save_path = '../data/bus_station_location.csv'
df.to_csv(save_path, index=False, encoding='utf-8-sig')

# 4. 연결 종료
conn.close()

print("CSV 저장 완료:", save_path)


/tmp/ipykernel_26024/1805133535.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CSV 저장 완료: ../data/bus_station_location.csv


---
## CSV 컬럼 추가 - bus_station_location.csv에 KIKmix.20190701.xlsx를 이용해서 행정동 칼럼 추가.
---


In [7]:
# 1. 파일 불러오기
df_bus = pd.read_csv('../data/bus_station_location.csv')  # 버스 위치 파일
df_kik = pd.read_excel('../data/KIKmix.20190701.xlsx', sheet_name='KIKmix')  # 행정동 매핑

# 2. 필요한 컬럼만 추출 및 이름 정리
df_kik_map = df_kik[['시군구명', '읍면동명', '동리명']].dropna()
df_kik_map = df_kik_map.rename(columns={
    '시군구명': 'gu',
    '읍면동명': '행정동',
    '동리명': 'dong'
})

# 3. gu + dong 기준으로 merge
df_merged = pd.merge(df_bus, df_kik_map, on=['gu', 'dong'], how='left')

# 4. 저장
df_merged.to_csv('../data/bus_station_location_with_admin.csv', index=False, encoding='utf-8-sig')


---
## CSV 추출 - bus_station_hourly_timeflow과 bus_station_location_with_admin를 이용해 행정동별 버스 승하차수 csv파일 만들기.
---


In [10]:
import pandas as pd

# 1. 파일 경로
flow_path = "../data/bus_station_hourly_timeflow.csv"
location_path = "../data/bus_station_location_with_admin.csv"

# 2. CSV 파일 불러오기
df_flow = pd.read_csv(flow_path)
df_location = pd.read_csv(location_path)

# 3. 정류장 ID 기준 merge
df_merged = pd.merge(df_flow, df_location, on='bus_station_id', how='left')

# 4. 승하차 합산
boarding_cols = [col for col in df_merged.columns if col.startswith('boarding_')]
alighting_cols = [col for col in df_merged.columns if col.startswith('alighting_')]
df_merged['total_passenger'] = df_merged[boarding_cols].sum(axis=1) + df_merged[alighting_cols].sum(axis=1)

# ✅ 5. 2024년 데이터만 필터링
df_merged_2024 = df_merged[df_merged['date_period'].astype(str).str.startswith("2024")]

# 6. 그룹화
df_grouped = df_merged_2024.groupby(['행정동', 'gu', 'date_period'], as_index=False)['total_passenger'].sum()

# 7. 저장
df_grouped.to_csv("../data/bus_hourly_by_admin_dong_2024.csv", index=False, encoding="utf-8-sig")

print("✅ 저장 완료: ../data/bus_hourly_by_admin_dong_2024.csv")


/tmp/ipykernel_26024/3880986186.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flow = pd.read_csv(flow_path)


✅ 저장 완료: ../data/bus_hourly_by_admin_dong_2024.csv
